In [1]:
import pandas as pd
import ms3 
from  ms3.utils import *
import os 
from ms3 import Score
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### Convert midi to mscx  
Convert a folder of midi files to the mscx format

In [90]:
path_midi = '/home/nulpe/Desktop/Tresillo/dataset/project_midi/tresillo/'
target = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos_billboard/'

dir_list = os.listdir(path_midi)

for el in dir_list:
    convert(path_midi+el, target+el[:-4]+'.mscx', MS='musescore3')

INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/tresillo/Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos_billboard/Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/tresillo/New rules-Dua Lipa.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos_billboard/New rules-Dua Lipa.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/tresillo/Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos_billboard/Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/tresillo/Willow-Taylor Swift.mid

### Mscx to rythm vectors including all instruments
Convert a folder of mscx files to a data frame of rythm vectors.  
Attetion: Every instrument has it's own rythm vector. To obtain oe rythm vector per song colapse per artist_song 

In [106]:
def rythm_vectors(in_dir, out_dir):
    list_sheet_music = os.listdir(in_dir)
    df_rythm_vectors =[]

    for idx, el in enumerate(list_sheet_music):
        if el[-4:] == 'mscx':

            
            #Get notes with onsets
            s = Score(dir_sheet_music+el)
            df = s.mscx.notes

            # Define instruments 
            instrument_dict = {}
            for key in s.mscx.metadata['parts']:
                for staff in s.mscx.metadata['parts'][key].keys():
                    instrument_dict[staff] = key


            #staff to instruments 
            df['instrument'] = [instrument_dict[el] if el in instrument_dict else 'na' for el in df.staff]


            # define quarter beat 
            df['quarter_beats'] = (df.mc_onset*16).astype('int32')


            #make rythm matrix & data frame
            df_histogram = df.groupby(['instrument','quarter_beats'])['mn'].agg(['count'])
            df_histogram = df_histogram.groupby(level=0).apply(lambda x: x.reset_index(level = 0).drop(['instrument'],axis=1).reindex(list(range(0,16)),fill_value=0).T)
            df_histogram = df_histogram.reset_index()

            df_histogram.insert(loc=0, column='song_artist', value=el[:-5])

            #concat to big rythm vector df
            if len(df_rythm_vectors) == 0: df_rythm_vectors = df_histogram

            df_rythm_vectors = pd.concat([df_rythm_vectors,df_histogram], axis=0)

    df_rythm_vectors.to_csv(out_dir, index = False)

    
dir_sheet_music = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos_billboard/'
out_dir = '/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv'
rythm_vectors(dir_sheet_music, out_dir)

In [107]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
df_rythm_vectors

,song_artist,instrument,level_1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"Eastside-benny blanco, Halsey, Khalid",Athmosphere,count,32,0,0,0,0,0,0,0,30,0,0,0,30,0,0,0
1,"Eastside-benny blanco, Halsey, Khalid",Bright Piano,count,64,0,0,0,0,0,0,0,32,0,0,0,60,0,0,0
2,"Eastside-benny blanco, Halsey, Khalid",Fingered Bass,count,16,0,0,16,16,0,0,0,15,0,0,15,15,0,7,0
3,"Eastside-benny blanco, Halsey, Khalid",Jazz Guitar,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
4,"Eastside-benny blanco, Halsey, Khalid",Nylon Gtr.,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Taylor Swift_willow,Gtr.Harmonics,count,12,0,0,4,0,0,4,0,4,0,0,3,0,0,3,0
95,Taylor Swift_willow,Ocarina,count,12,0,0,4,0,0,4,0,4,0,0,3,0,0,3,0
96,Taylor Swift_willow,Percussion,count,0,0,3,0,0,0,4,0,0,0,3,0,0,0,3,0
97,Taylor Swift_willow,Slap Bass 1,count,75,0,0,20,0,0,20,0,60,0,5,20,5,0,15,0


### Calculate cosine similarity between tresillo vecetors and songs 

In [108]:
def collapse_normalize_vectors(df_rythm_vectors):
    rythm_vector_collaped = df_rythm_vectors.groupby(['song_artist']).agg(['sum'])
    rythm_vector_collaped.columns = rythm_vector_collaped.columns.droplevel(1)
    rythm_vector_collaped = rythm_vector_collaped.drop(['instrument', 'level_1'],axis=1)
    rythm_vector_collaped["sum"] = rythm_vector_collaped.sum(axis=1)
    rythm_vector_collaped = rythm_vector_collaped.loc[:,"0":"15"].div(rythm_vector_collaped["sum"], axis=0)
    return rythm_vector_collaped

def calc_cosine_sim(rythm_vectors, tresillo_vector):
    rythm_vectors['cosine_sim_tresillo'] = cosine_similarity(rythm_vectors.loc[:,"0":"15"],tresillo_vector)
    return rythm_vectors

In [109]:
# the billboard tresillo vectors 
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
rythm_vector_collaped = collapse_normalize_vectors(df_rythm_vectors)
rythm_vector_collaped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
song_artist,,,,,,,,,,,,,,,,
"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",0.211900,0.007307,0.015136,0.102296,0.062109,0.000000,0.105428,0.000000,0.199374,0.008873,0.016701,0.103862,0.067328,0.001566,0.096033,0.002088
"Eastside-benny blanco, Halsey, Khalid",0.221595,0.006351,0.019054,0.064220,0.042343,0.014820,0.052929,0.010586,0.172900,0.007763,0.019760,0.062103,0.187721,0.022583,0.062809,0.032463
"Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber",0.151248,0.004607,0.122073,0.013820,0.119386,0.004607,0.099808,0.002687,0.110173,0.003071,0.101727,0.007678,0.122073,0.008829,0.116699,0.011516
New rules-Dua Lipa,0.176933,0.020533,0.050240,0.092180,0.061162,0.014417,0.050677,0.010922,0.097859,0.015291,0.076453,0.121014,0.082569,0.015291,0.102665,0.011796
"Rockabye-Clean Bandit, Sean Paul, Anne-Marie",0.183642,0.003086,0.052469,0.075617,0.043981,0.008488,0.075617,0.037037,0.154321,0.033951,0.038580,0.090278,0.053241,0.023148,0.105710,0.020833
Shape of you-Ed Sheran,0.148522,0.023792,0.031723,0.085797,0.051911,0.026676,0.121846,0.008652,0.144196,0.020908,0.054074,0.080750,0.054074,0.019466,0.120404,0.007210
Taylor Swift_willow,0.299113,0.000000,0.003802,0.081115,0.070976,0.000000,0.086185,0.000000,0.221800,0.000000,0.019011,0.076046,0.077313,0.000000,0.064639,0.000000
The+Chainsmokers+-+Don't+Let+Me+Down,0.181953,0.000000,0.029586,0.005917,0.125740,0.000000,0.100592,0.000000,0.170118,0.000000,0.059172,0.047337,0.184911,0.000000,0.094675,0.000000
Treat you better-Shawn Mendes,0.150000,0.023656,0.036559,0.086559,0.064516,0.024731,0.085484,0.023118,0.138710,0.023118,0.029570,0.087634,0.067204,0.032796,0.095699,0.030645


In [110]:
# the self defined synthetique tresillo vectors 
df_synt_tresillo = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillio.csv')
df_synt_tresillo = df_synt_tresillo[df_synt_tresillo['song_artist']!='Raggetone'].drop(['instrument','level_1'], axis=1)
df_synt_tresillo.index = df_synt_tresillo.song_artist
df_synt_tresillo.drop(['song_artist'], axis=1)
df_synt_tresillo["sum"] = df_synt_tresillo.sum(axis=1)
df_synt_tresillo = df_synt_tresillo.loc[:,"0":"15"].div(df_synt_tresillo["sum"], axis=0)
tresillo_vector = df_synt_tresillo.sum()/len(df_synt_tresillo)
tresillo_vector = np.asarray(tresillo_vector).reshape(-1, 1).T

In [114]:
# all synthetique tresillos collapsed together, not very convincing results, it seems it averages the effect out, especially if you compare it to not tresillo songs
rythm_vector_collaped = calc_cosine_sim(rythm_vector_collaped, tresillo_vector)
rythm_vector_collaped['cosine_sim_tresillo']

song_artist
Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul               0.913635
Eastside-benny blanco, Halsey, Khalid                       0.803275
Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber    0.774479
New rules-Dua Lipa                                          0.926529
Rockabye-Clean Bandit, Sean Paul, Anne-Marie                0.915665
Shape of you-Ed Sheran                                      0.964258
Taylor Swift_willow                                         0.828289
The+Chainsmokers+-+Don't+Let+Me+Down                        0.779599
Treat you better-Shawn Mendes                               0.940530
Willow-Taylor Swift                                         0.903536
Name: cosine_sim_tresillo, dtype: float64

In [112]:
# not tresillo songs vectors
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_no_tresillos_4_4.csv')
non_tresillio_vectors = collapse_normalize_vectors(df_rythm_vectors)
non_tresillio_vectors = calc_cosine_sim(non_tresillio_vectors, tresillo_vector)
non_tresillio_vectors['cosine_sim_tresillo']

song_artist
A hard days night-Beatles               0.794264
Cocaine-Eric Clapton                    0.789114
Come together-Beatles                   0.778343
Come_As_You_Are-Nirvana                 0.771313
Crazy little thing called love-Queen    0.718345
Hey Jude- Beatles                       0.697940
I Shot the Sheriff-Eric Clapton         0.909999
Walk of Life- Dire Straits              0.708941
Name: cosine_sim_tresillo, dtype: float64

In [113]:
#lets compare all self defined tresillos to tresillo billboard songs and not tresillo billboard songs ---> vanilla tresillo reflects the most our intuition
for index in df_synt_tresillo.index:
    print('\n')
    print(index)
    vector = np.asarray(df_synt_tresillo.loc[index,"0":"15"]).reshape(-1, 1).T
    print('tresillio songs')
    print(calc_cosine_sim(rythm_vector_collaped, vector)['cosine_sim_tresillo'])
    print('no tresillio songs')
    print(calc_cosine_sim(non_tresillio_vectors, vector)['cosine_sim_tresillo'])



Cinquillo
tresillio songs
song_artist
Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul               0.732785
Eastside-benny blanco, Halsey, Khalid                       0.616876
Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber    0.691407
New rules-Dua Lipa                                          0.808246
Rockabye-Clean Bandit, Sean Paul, Anne-Marie                0.804314
Shape of you-Ed Sheran                                      0.842578
Taylor Swift_willow                                         0.645638
The+Chainsmokers+-+Don't+Let+Me+Down                        0.587273
Treat you better-Shawn Mendes                               0.815995
Willow-Taylor Swift                                         0.785162
Name: cosine_sim_tresillo, dtype: float64
no tresillio songs
song_artist
A hard days night-Beatles               0.698336
Cocaine-Eric Clapton                    0.647976
Come together-Beatles                   0.676899
Come_As_You_Are-Nirvana                 0.645211
C

In [121]:
#vanilla tresillo reflects the most our intuition
vanilla_tresillo_vector = np.asarray(df_synt_tresillo.loc['Vanilla_Tresillo',"0":"15"]).reshape(-1, 1).T

print('tresillio songs')
print(calc_cosine_sim(rythm_vector_collaped, vanilla_tresillo_vector)['cosine_sim_tresillo'])
print('no tresillio songs')
print(calc_cosine_sim(non_tresillio_vectors, vanilla_tresillo_vector)['cosine_sim_tresillo'])

tresillio songs
song_artist
Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul               0.908945
Eastside-benny blanco, Halsey, Khalid                       0.711226
Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber    0.605146
New rules-Dua Lipa                                          0.838867
Rockabye-Clean Bandit, Sean Paul, Anne-Marie                0.880677
Shape of you-Ed Sheran                                      0.915565
Taylor Swift_willow                                         0.811188
The+Chainsmokers+-+Don't+Let+Me+Down                        0.660548
Treat you better-Shawn Mendes                               0.883772
Willow-Taylor Swift                                         0.760835
Name: cosine_sim_tresillo, dtype: float64
no tresillio songs
song_artist
A hard days night-Beatles               0.639260
Cocaine-Eric Clapton                    0.656375
Come together-Beatles                   0.626717
Come_As_You_Are-Nirvana                 0.619208
Crazy little 

tresillio songs mean tresillo-ness:  0.7976769323047527

no tresillio songs mean tresillo-ness:  0.63573893650337


### Use bootstrapping to obtain upper and lower 97.5% confidence interval 

In [123]:
def bootstrap_CI(data, nbr_draws):
    means = np.zeros(nbr_draws)
    data = np.array(data)

    for n in range(nbr_draws):
        indices = np.random.randint(0, len(data), len(data))
        data_tmp = data[indices] 
        means[n] = np.nanmean(data_tmp)

    return [np.nanpercentile(means, 2.5),np.nanpercentile(means, 97.5)]


lower_ci, upper_ci

(0.741500345800673, 0.8515609015400654)

In [125]:
lower_tresillo_ci, upper_tresillo_ci = bootstrap_CI(calc_cosine_sim(rythm_vector_collaped, vanilla_tresillo_vector)['cosine_sim_tresillo'], 100)
lower_no_tresillo_ci, upper_no_tresillo_ci = bootstrap_CI(calc_cosine_sim(non_tresillio_vectors, vanilla_tresillo_vector)['cosine_sim_tresillo'], 100)

mean_tresillo = calc_cosine_sim(rythm_vector_collaped, vanilla_tresillo_vector)['cosine_sim_tresillo'].mean()
mean_no_tresillo = calc_cosine_sim(non_tresillio_vectors, vanilla_tresillo_vector)['cosine_sim_tresillo'].mean()
print('tresillio songs mean tresillo-ness: ', mean_tresillo)
print('tresillo upper and lower ci on 100 draws: ', lower_tresillo_ci, upper_tresillo_ci)
print('\n')
print('no tresillio songs mean tresillo-ness: ', mean_no_tresillo)
print('no tresillo upper and lower ci on 100 draws: ', lower_no_tresillo_ci, upper_no_tresillo_ci)

tresillio songs mean tresillo-ness:  0.7976769323047527
tresillo upper and lower ci on 100 draws:  0.7205627050065968 0.8591490509117414


no tresillio songs mean tresillo-ness:  0.63573893650337
no tresillo upper and lower ci on 100 draws:  0.6037983392784003 0.6704955900495142
